In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta, date
import geopandas as gpd
from shapely.geometry import Point

In [2]:
df_acidentes_raw = pd.read_csv('data/Acidentes/sinistros_com_chuva_2022-2025.csv', delimiter=';')

### Limpeza dos dados

In [3]:
# Limpeza
df_acidentes_raw['latitude'] = df_acidentes_raw['latitude'].str.replace(',', '.').astype(float)
df_acidentes_raw['longitude'] = df_acidentes_raw['longitude'].str.replace(',', '.').astype(float)
df_acidentes_raw['data_sinistro'] = pd.to_datetime(df_acidentes_raw['data_sinistro'], dayfirst=True)
df_acidentes_raw['hora_sinistro'] = df_acidentes_raw['hora_sinistro'].astype(str).replace('99:99', np.nan)
df_acidentes_raw['hora_sinistro_dt'] = pd.to_datetime(df_acidentes_raw['hora_sinistro'], format='%H:%M')


# Mapear 'tipo_via' para números (0: Municipal, 1: Rodovia)
tipo_via_map = {'VIAS MUNICIPAIS': 0, 'RODOVIAS': 1}
df_acidentes_raw['tipo_via_num'] = df_acidentes_raw['tipo_via'].map(tipo_via_map)

# Mapear 'Chuva' para binário (0: Sem chuva, 1: Com chuva)
df_acidentes_raw['Chuva'] = df_acidentes_raw['Chuva'].apply(lambda x: 1 if x != 'Sem chuva' else 0)

# Extrair data e hora
df_acidentes_raw['hora'] = df_acidentes_raw['hora_sinistro_dt'].dt.hour
df_acidentes_raw.fillna({'hora': -1}, inplace=True)
df_acidentes_raw['hora'] = df_acidentes_raw['hora'].astype(int)
df_acidentes_raw['data'] = df_acidentes_raw['data_sinistro'].dt.date
df_acidentes_raw['dia_semana'] = df_acidentes_raw['data_sinistro'].dt.dayofweek
df_acidentes_raw['mes'] = df_acidentes_raw['data_sinistro'].dt.month
df_acidentes_raw['is_weekend'] = df_acidentes_raw['dia_semana'].isin([5, 6]).astype(int)

# Definir a classe positiva
df_acidentes_raw['Sinistro'] = 1

colunas_veiculos = [col for col in df_acidentes_raw.columns if 'tp_veiculo' in col]
colunas_base = ['latitude', 'longitude', 'data', 'dia_semana', 'mes', 'is_weekend', 'hora', 'Sinistro', 'Chuva', 'tipo_via_num']
df_positivos = df_acidentes_raw[colunas_base + colunas_veiculos]

print(f"Total de positivos: {len(df_positivos)}")
df_positivos

Total de positivos: 8189


,latitude,longitude,data,dia_semana,mes,is_weekend,hora,Sinistro,Chuva,tipo_via_num,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.318170,-49.114148,2022-01-01,5,1,1,4,1,0,0.0,0,0,0,0,0,0,0
1,-22.305238,-49.097946,2022-01-01,5,1,1,4,1,0,0.0,0,0,0,0,0,0,0
2,-22.306460,-49.104453,2022-01-01,5,1,1,4,1,0,0.0,0,0,1,0,0,0,0
3,-22.312349,-49.122265,2022-01-02,6,1,1,19,1,0,0.0,0,0,0,0,0,0,0
4,-22.338166,-49.053244,2022-01-02,6,1,1,20,1,0,0.0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,-22.391376,-49.069106,2025-02-28,4,2,0,22,1,0,0.0,0,0,0,0,0,0,3
8185,-22.295708,-49.045586,2025-02-28,4,2,0,13,1,0,0.0,0,0,1,0,0,0,1
8186,-22.338551,-49.069617,2025-02-28,4,2,0,13,1,0,0.0,0,0,0,0,0,0,2
8187,-22.305703,-49.051948,2025-02-28,4,2,0,18,1,0,0.0,0,0,0,0,0,0,1


### Função temporal

In [4]:
# Criamos o conjunto de busca para checagem de colisão (muito rápido)
lookup_acidentes = set(zip(
    df_positivos['latitude'],
    df_positivos['longitude'],
    df_positivos['data'], # A coluna 'data' já é um objeto date
    df_positivos['hora']
))

# Pega a lista de colunas de veículos para usar depois
colunas_veiculos = [col for col in df_positivos.columns if 'tp_veiculo' in col]

def gerar_features_temporais_aleatorias():
    start_date = date(2022, 1, 1)
    end_date = date(2025, 12, 31)
    total_days = (end_date - start_date).days
    
    data_aleatoria = start_date + timedelta(days=random.randint(0, total_days))
    hora_aleatoria = random.randint(0, 23)
    dia_semana_aleatorio = data_aleatoria.weekday() # Segunda=0, Domingo=6
    
    features = {
        'data': data_aleatoria,
        'hora': hora_aleatoria,
        'dia_semana': dia_semana_aleatorio,
        'mes': data_aleatoria.month,
        'is_weekend': 1 if dia_semana_aleatorio >= 5 else 0
    }
    return features

def sortear_chuva(probabilidade_chuva=0.2):
    return 1 if random.random() < probabilidade_chuva else 0

### Geração dos Negativos Tipo 1 (Mesmo Local; Data, Hora e Chuva Aleatório)

In [5]:
negativos_tipo1_lista = []
total_positivos = len(df_positivos)

print("Gerando Negativos Tipo 1 (Contraste Temporal)...")
for index, acidente in df_positivos.iterrows():
    lat = acidente['latitude']
    lon = acidente['longitude']
    
    # Loop para garantir que o novo ponto sorteado não seja um acidente real
    while True:
        features_temporais = gerar_features_temporais_aleatorias()
        # Checa se a combinação (local, data, hora) já existe nos acidentes
        if (lat, lon, features_temporais['data'], features_temporais['hora']) not in lookup_acidentes:
            break # Se não existe, é um negativo válido
            
    # Cria o dicionário do novo negativo com todas as colunas
    novo_negativo = {
        'latitude': lat,
        'longitude': lon,
        'tipo_via_num': acidente['tipo_via_num'], # Mantém o tipo de via do local
        'Chuva': sortear_chuva(),
        'Sinistro': 0
    }
    
    # Adiciona as novas features temporais
    novo_negativo.update(features_temporais)
    
    # Adiciona as colunas de veículo com valor 0
    for col in colunas_veiculos:
        novo_negativo[col] = 0
        
    negativos_tipo1_lista.append(novo_negativo)

df_negativos_tipo1 = pd.DataFrame(negativos_tipo1_lista)
print("\nNegativos Tipo 1 Gerados com Sucesso.")
print(f"Total de negativos tipo 1: {len(df_negativos_tipo1)}")
df_negativos_tipo1

Gerando Negativos Tipo 1 (Contraste Temporal)...

Negativos Tipo 1 Gerados com Sucesso.
Total de negativos tipo 1: 8189


,latitude,longitude,tipo_via_num,Chuva,Sinistro,data,hora,dia_semana,mes,is_weekend,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.318170,-49.114148,0.0,0,0,2025-12-10,3,2,12,0,0,0,0,0,0,0,0
1,-22.305238,-49.097946,0.0,0,0,2024-07-27,22,5,7,1,0,0,0,0,0,0,0
2,-22.306460,-49.104453,0.0,0,0,2024-03-13,13,2,3,0,0,0,0,0,0,0,0
3,-22.312349,-49.122265,0.0,0,0,2023-10-12,22,3,10,0,0,0,0,0,0,0,0
4,-22.338166,-49.053244,0.0,1,0,2023-03-30,11,3,3,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,-22.391376,-49.069106,0.0,0,0,2023-12-12,19,1,12,0,0,0,0,0,0,0,0
8185,-22.295708,-49.045586,0.0,0,0,2025-08-19,18,1,8,0,0,0,0,0,0,0,0
8186,-22.338551,-49.069617,0.0,0,0,2022-09-15,21,3,9,0,0,0,0,0,0,0,0
8187,-22.305703,-49.051948,0.0,0,0,2023-04-17,18,0,4,0,0,0,0,0,0,0,0


### Geração dos Negativos Tipo 2 (Local Aleatório; Data, Hora e tempo Aleatório)

In [6]:
gdf_ruas = gpd.read_file('ruas_de_bauru.gpkg')

# Mapeia os tipos de via do OpenStreetMap para o padrão (0 ou 1)
highway_map = {
     'motorway': 1, 'trunk': 1, 'primary': 0, 'secondary': 0,
     'tertiary': 0, 'residential': 0, 'unclassified': 0
}

gdf_ruas['tipo_via_num'] = gdf_ruas['highway'].map(highway_map).fillna(0).astype(int)

min_lon, min_lat, max_lon, max_lat = gdf_ruas.total_bounds
uniao_ruas = gdf_ruas.union_all().buffer(0.0001)

negativos_tipo2_lista = []
num_negativos_tipo2 = len(df_positivos)

print("\nGerando Negativos Tipo 2 (Contraste Espacial)...")
while len(negativos_tipo2_lista) < num_negativos_tipo2:
    lon_aleatorio = random.uniform(min_lon, max_lon)
    lat_aleatoria = random.uniform(min_lat, max_lat)
    ponto_aleatorio = Point(lon_aleatorio, lat_aleatoria)
    
    if ponto_aleatorio.within(uniao_ruas):
        indices_proximos = gdf_ruas.sindex.nearest(ponto_aleatorio)
        indice_da_rua = indices_proximos[1][0] # Extrai o número inteiro
        rua_mais_proxima = gdf_ruas.iloc[[indice_da_rua]]
        
        tipo_via_ponto = rua_mais_proxima['tipo_via_num'].iloc[0]
        
        features_temporais = gerar_features_temporais_aleatorias()
        
        novo_negativo = {
            'latitude': lat_aleatoria,
            'longitude': lon_aleatorio,
            'tipo_via_num': tipo_via_ponto,
            'Chuva': sortear_chuva(),
            'Sinistro': 0
        }
        novo_negativo.update(features_temporais)
        for col in colunas_veiculos:
            novo_negativo[col] = 0
            
        negativos_tipo2_lista.append(novo_negativo)

df_negativos_tipo2 = pd.DataFrame(negativos_tipo2_lista)
print(f"Gerado {len(negativos_tipo2_lista)}/{num_negativos_tipo2} negativos tipo 2...")
print("\nNegativos Tipo 2 Gerados com Sucesso.")
df_negativos_tipo2


Gerando Negativos Tipo 2 (Contraste Espacial)...
Gerado 8189/8189 negativos tipo 2...

Negativos Tipo 2 Gerados com Sucesso.


,latitude,longitude,tipo_via_num,Chuva,Sinistro,data,hora,dia_semana,mes,is_weekend,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.354833,-49.070042,0,0,0,2023-02-09,12,3,2,0,0,0,0,0,0,0,0
1,-22.337714,-48.955283,0,0,0,2022-07-02,5,5,7,1,0,0,0,0,0,0,0
2,-22.319247,-49.154072,1,0,0,2024-01-19,4,4,1,0,0,0,0,0,0,0,0
3,-22.214620,-49.061520,0,0,0,2022-08-19,18,4,8,0,0,0,0,0,0,0,0
4,-22.329015,-49.114144,0,0,0,2022-05-16,2,0,5,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,-22.366109,-49.028989,0,1,0,2024-02-27,15,1,2,0,0,0,0,0,0,0,0
8185,-22.322965,-49.130605,0,0,0,2023-10-28,9,5,10,1,0,0,0,0,0,0,0
8186,-22.316827,-49.066807,0,1,0,2025-03-25,23,1,3,0,0,0,0,0,0,0,0
8187,-22.319553,-49.009552,0,0,0,2023-12-16,13,5,12,1,0,0,0,0,0,0,0


### Unir, embaralhar e salvar

In [8]:
df_final = pd.concat([df_positivos, df_negativos_tipo1], ignore_index=True)
df_final = pd.concat([df_final, df_negativos_tipo2], ignore_index=True)

# Embaralha o dataset final para garantir aleatoriedade
df_final = df_final.sample(frac=1).reset_index(drop=True)

df_final.to_csv('dataset_final_para_modelo.csv', index=False, decimal=',')

print(f"Total de amostras: {len(df_final)}")
print("Distribuição das classes:")
print(df_final['Sinistro'].value_counts())
df_final

Total de amostras: 24567
Distribuição das classes:
Sinistro
0    16378
1     8189
Name: count, dtype: int64


,latitude,longitude,data,dia_semana,mes,is_weekend,hora,Sinistro,Chuva,tipo_via_num,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.370454,-49.051246,2025-09-01,0,9,0,2,0,0,1.0,0,0,0,0,0,0,0
1,-22.371995,-49.025240,2025-08-22,4,8,0,11,0,0,0.0,0,0,0,0,0,0,0
2,-22.340207,-49.114017,2025-12-02,1,12,0,23,0,0,0.0,0,0,0,0,0,0,0
3,-22.332466,-49.056129,2024-04-23,1,4,0,17,0,1,0.0,0,0,0,0,0,0,0
4,-22.323449,-49.058255,2023-07-26,2,7,0,7,1,0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24562,-22.399218,-49.016150,2023-01-03,1,1,0,11,0,1,1.0,0,0,0,0,0,0,0
24563,-22.315863,-49.043546,2022-08-18,3,8,0,22,1,0,0.0,0,0,0,0,0,0,0
24564,-22.336420,-49.100087,2024-04-19,4,4,0,6,1,0,0.0,0,0,0,0,0,0,0
24565,-22.304959,-49.098589,2022-09-24,5,9,1,17,1,0,0.0,0,0,0,0,0,0,0
